In [1]:
import pandas as pd

# Đọc file CSV
df = pd.read_csv("archive\imdb-movies-dataset.csv")

# Xem 5 dòng đầu
df.head()


<>:4: SyntaxWarning: invalid escape sequence '\i'
<>:4: SyntaxWarning: invalid escape sequence '\i'
C:\Users\Admin\AppData\Local\Temp\ipykernel_36408\2021598301.py:4: SyntaxWarning: invalid escape sequence '\i'
  df = pd.read_csv("archive\imdb-movies-dataset.csv")


,Poster,Title,Year,Certificate,Duration (min),Genre,Rating,Metascore,Director,Cast,Votes,Description,Review Count,Review Title,Review
0,https://m.media-amazon.com/images/M/MV5BYWRkZj...,The Idea of You,2023.0,R,115.0,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","28,744","Solène, a 40-year-old single mom, begins an un...",166,Hypocrisy as an idea,"This film, as well as the reaction to it, is a..."
1,https://m.media-amazon.com/images/M/MV5BZGI4NT...,Kingdom of the Planet of the Apes,2023.0,PG-13,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","22,248","Many years after the reign of Caesar, a young ...",183,A phenomenal start to another trilogy!,"I'm a big fan of all the planet of the apes, a..."
2,https://m.media-amazon.com/images/M/MV5BZjIyOT...,Unfrosted,2023.0,PG-13,97.0,"Biography, Comedy, History",5.5,42.0,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...","18,401","In 1963 Michigan, business rivals Kellogg's an...",333,not funny,Pretty much the worst criticism you can lay on...
3,https://m.media-amazon.com/images/M/MV5BMjA5Zj...,The Fall Guy,2023.0,PG-13,126.0,"Action, Comedy, Drama",7.3,73.0,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...","38,953",A down-and-out stuntman must find the missing ...,384,Everything you needed and more!,Just got out of the Austin premier at SXSW and...
4,https://m.media-amazon.com/images/M/MV5BNTk1MT...,Challengers,2023.0,R,131.0,"Drama, Romance, Sport",7.7,82.0,Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...","32,517","Tashi, a former tennis prodigy turned coach, t...",194,"Watch ""Match Point"" instead",This is a tough one. I liked the concept and t...


# 📊 **DATA OVERVIEW**
---

This section provides a brief overview of the dataset, including the number of movies, features, and missing values.

- **Total movies:** 10,000
- **Features:** Title, Year, Certificate, Duration, Genre, Rating, Metascore, Director, Cast
- **Missing values:** Some columns contain NaN values that need preprocessing


In [2]:
import plotly.express as px

total_movies = len(df)

fig = px.bar(
    x=["Total Movies"],
    y=[total_movies],
    text=[total_movies],
    labels={"x": "Movies", "y": "Count"},
    title="Total Number of Movies in the Dataset"
)

fig.update_traces(textposition="inside") #
fig.show()


### 🧹 **Remove Duplicates**
---

This step involves identifying and removing duplicate entries in the dataset to ensure data quality.  
Duplicate movies (with the same Title, Year, or other key features) are dropped to prevent bias in the recommendation system.


In [3]:
df = df.drop_duplicates()
total_movies = len(df)

fig = px.bar(
    x=["Total Movies"],
    y=[total_movies],
    text=[total_movies],
    labels={"x": "Movies", "y": "Count"},
    title="Total Number of Movies in the Dataset"
)

fig.update_traces(textposition="inside") #
fig.show()

### ⚠️ **Missing Values**
---

This step identifies and handles missing values (NaN) in the dataset.  
Columns with missing data need proper treatment to ensure the recommendation system works correctly.  


In [4]:
import pandas as pd
import plotly.express as px

# Count NaN per column
nan_counts = df.isna().sum().reset_index()
nan_counts.columns = ["Column", "NaN_Count"]

# Visualization
fig = px.bar(
    nan_counts,
    x="Column",
    y="NaN_Count",
    text="NaN_Count",
    title="Number of Missing (NaN) Values per Column",
    labels={"Column": "Column", "NaN_Count": "Missing Values"}
)

fig.update_traces(textposition="outside")
fig.update_layout(xaxis_tickangle=45)

fig.update_layout(yaxis=dict(range=[0, 3000]))  

fig.show()


# Summary of Feature Processing

The dataset contains multiple types of features, each processed differently for Machine Learning:
These processed features are then concatenated into a final feature matrix, combining:
- Numeric features
- One-hot categorical features
- TF-IDF sparse vectors
- Dense embeddings


#### 1️. One-Hot Encoding (Categorical)
- **Columns:** `Certificate`, `Year`
- **Method:** Convert categories into one-hot vectors.
- **Purpose:** Transform categorical values into numerical features for models.

In [5]:
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

import plotly.express as px
import pandas as pd
import numpy as np

# Chuyển Year sang số
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

# Loại bỏ NaN
df_year = df.dropna(subset=['Year'])

# Vẽ histogram theo từng 10 năm
fig = px.histogram(
    df_year,
    x='Year',
    title='Distribution of Movies by Year (10-year bins)',
    labels={'Year': 'Year', 'count': 'Number of Movies'},
)

# Set bin size = 10 năm
fig.update_traces(
    xbins=dict(start=df_year['Year'].min(), 
               end=df_year['Year'].max(), 
               size=10)
)

fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Number of Movies'
)

fig.show()


In [6]:
def year_group(year):
    # chuyển về số, lỗi thì thành NaN
    year = pd.to_numeric(year, errors='coerce')

    if pd.isna(year):
        return 'Unknown'

    year = int(year)

    if year <= 1945:
        return 'Pre-1945'
    elif 1946 <= year <= 1959:
        return '1946-1959'
    elif 1960 <= year <= 1969:
        return '1960-1969'
    elif 1970 <= year <= 1979:
        return '1970-1979'
    elif 1980 <= year <= 1989:
        return '1980-1989'
    elif 1990 <= year <= 1999:
        return '1990-1999'
    elif 2000 <= year <= 2009:
        return '2000-2009'
    else:
        return '2010+'
    

df['Year_Group'] = df['Year'].apply(year_group)

year_ohe = pd.get_dummies(df['Year_Group'], prefix='year')

df = pd.concat([df, year_ohe], axis=1).drop(columns=['Year_Group'])

df.head()


,Poster,Title,Year,Certificate,Duration (min),Genre,Rating,Metascore,Director,Cast,...,Review,year_1946-1959,year_1960-1969,year_1970-1979,year_1980-1989,year_1990-1999,year_2000-2009,year_2010+,year_Pre-1945,year_Unknown
0,https://m.media-amazon.com/images/M/MV5BYWRkZj...,The Idea of You,2023.0,R,115.0,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...",...,"This film, as well as the reaction to it, is a...",False,False,False,False,False,False,True,False,False
1,https://m.media-amazon.com/images/M/MV5BZGI4NT...,Kingdom of the Planet of the Apes,2023.0,PG-13,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...",...,"I'm a big fan of all the planet of the apes, a...",False,False,False,False,False,False,True,False,False
2,https://m.media-amazon.com/images/M/MV5BZjIyOT...,Unfrosted,2023.0,PG-13,97.0,"Biography, Comedy, History",5.5,42.0,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...",...,Pretty much the worst criticism you can lay on...,False,False,False,False,False,False,True,False,False
3,https://m.media-amazon.com/images/M/MV5BMjA5Zj...,The Fall Guy,2023.0,PG-13,126.0,"Action, Comedy, Drama",7.3,73.0,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...",...,Just got out of the Austin premier at SXSW and...,False,False,False,False,False,False,True,False,False
4,https://m.media-amazon.com/images/M/MV5BNTk1MT...,Challengers,2023.0,R,131.0,"Drama, Romance, Sport",7.7,82.0,Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...",...,This is a tough one. I liked the concept and t...,False,False,False,False,False,False,True,False,False


In [7]:
import pandas as pd
import plotly.express as px

# Replace NaN/None/empty with 'Unknown'
df["Certificate_clean"] = df["Certificate"].fillna("NaN")
df["Certificate_clean"] = df["Certificate_clean"].replace("", "NaN")

# Count each certificate
cert_counts = df["Certificate_clean"].value_counts().reset_index()
cert_counts.columns = ["Certificate", "Count"]

# Visualization
fig = px.bar(
    cert_counts,
    x="Certificate",
    y="Count",
    text="Count",
    title="Number of Movies by Certificate (Including Unknown)",
    labels={"Certificate": "Certificate", "Count": "Count"}
)

fig.update_traces(textposition="outside")
fig.update_layout(xaxis_tickangle=45)

# Automatically raise y-axis so bar never overlaps
y_max = cert_counts["Count"].max()
fig.update_layout(yaxis=dict(range=[0, y_max + y_max*0.2]))  

fig.show()

# Remove the temporary column
df.drop(columns=["Certificate_clean"], inplace=True)


In [8]:
import pandas as pd

# 1. Replace NaN or empty entries with "Unknown"
df["Certificate"] = df["Certificate"].fillna("Unknown")
df["Certificate"] = df["Certificate"].replace("", "Unknown")

# 2. One-hot encode the Certificate column
cert_ohe = pd.get_dummies(df["Certificate"], prefix="cert")

# 3. Concatenate one-hot columns back into the main DataFrame
df = pd.concat([df, cert_ohe], axis=1)

df.head()


,Poster,Title,Year,Certificate,Duration (min),Genre,Rating,Metascore,Director,Cast,...,cert_U,cert_U/A,cert_U/A 16+,cert_UA,cert_UA 13+,cert_UA 16+,cert_UA 7+,cert_Unknown,cert_Unrated,cert_X
0,https://m.media-amazon.com/images/M/MV5BYWRkZj...,The Idea of You,2023.0,R,115.0,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...",...,False,False,False,False,False,False,False,False,False,False
1,https://m.media-amazon.com/images/M/MV5BZGI4NT...,Kingdom of the Planet of the Apes,2023.0,PG-13,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...",...,False,False,False,False,False,False,False,False,False,False
2,https://m.media-amazon.com/images/M/MV5BZjIyOT...,Unfrosted,2023.0,PG-13,97.0,"Biography, Comedy, History",5.5,42.0,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...",...,False,False,False,False,False,False,False,False,False,False
3,https://m.media-amazon.com/images/M/MV5BMjA5Zj...,The Fall Guy,2023.0,PG-13,126.0,"Action, Comedy, Drama",7.3,73.0,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...",...,False,False,False,False,False,False,False,False,False,False
4,https://m.media-amazon.com/images/M/MV5BNTk1MT...,Challengers,2023.0,R,131.0,"Drama, Romance, Sport",7.7,82.0,Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...",...,False,False,False,False,False,False,False,False,False,False


#### 2️. Median Imputation (Numeric)
- **Columns:** `Rating`, `Duration (min)`, `Votes`, `Review Count`
- **Method:** Fill missing values with the median of the column.
- **Purpose:** Handle missing numeric data without biasing the distribution.


In [9]:
import plotly.express as px
import numpy as np
import pandas as pd

# Chuyển Duration sang int
df['Duration_int'] = df['Duration (min)'].dropna().astype(int)

# Tạo các bin cách nhau 30 phút
bins = np.arange(0, df['Duration_int'].max() + 45, 45)

fig = px.histogram(
    df,
    x='Duration_int',
    title='Distribution of Movie Duration (Bin width = 45 min)',
    labels={'Duration_int': 'Duration (min)', 'count': 'Number of Movies'}
)

# Cập nhật trục x theo bin 30 phút
fig.update_xaxes(dtick=45)

fig.show()

# Điền NaN bằng median nếu cần
df['Rating_clean'] = df['Rating'].fillna(df['Rating'].median())

# Lọc bỏ 0
df_hist = df[df['Rating_clean'] > 0]

# Vẽ histogram
fig = px.histogram(
    df_hist,
    x='Rating_clean',
    nbins=10,  # từ 1 đến 10, mỗi bin 1
    title='Distribution of Movie Ratings (1 to 10)',
    labels={'Rating_clean': 'Rating', 'count': 'Number of Movies'}
)

# Hiển thị số lượng trên cột, thêm đơn vị 'movies'
fig.update_traces(texttemplate='%{y} movies', textposition='outside')

# Lấy giá trị max thực tế trong histogram
hist_values = np.histogram(df_hist['Rating_clean'], bins=10)[0]
y_max = hist_values.max()

fig.update_layout(
    xaxis_title='Rating',
    yaxis_title='Number of Movies',
    yaxis=dict(range=[0, y_max * 1.5]),  
)

fig.show()

In [10]:
import pandas as pd

# Fill missing values in 'Duration' with median
df['Duration (min)'] = df['Duration (min)'].fillna(df['Duration (min)'].median())

df['Rating'] = df['Rating'].fillna(df['Rating'].median())

In [11]:
# --- Clean numeric columns that contain commas ---
numeric_cols = ["Votes", "Review Count"]

for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# --- Fill NaN using median ---
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

#### 3️. TF-IDF (Text: Multi-label)
- **Columns:** `Genre`
- **Method:** 
  - Replace NaN with empty string.
  - Replace commas with spaces to separate genre words.
  - Apply TF-IDF vectorization to convert text into numeric features.
- **Purpose:** Capture textual information about genres as sparse vectors for ML models.

In [12]:
# --- TF-IDF cho GENRE ---

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# 1. Làm sạch Genre
df["Genre_clean"] = df["Genre"].fillna("").astype(str)

# Chuyển "Action, Adventure" → "Action Adventure"
df["Genre_clean"] = df["Genre_clean"].str.replace(",", " ", regex=False)

# 2. TF-IDF vectorizer (giới hạn 32 feature cho nhẹ)
tfidf = TfidfVectorizer(max_features=32)
genre_tfidf = tfidf.fit_transform(df["Genre_clean"])

print("TF-IDF shape:", genre_tfidf.shape)

# 3. Đưa TF-IDF vào DataFrame
genre_tfidf_df = pd.DataFrame(
    genre_tfidf.toarray(),
    columns=[f"genre_vec_{i}" for i in range(genre_tfidf.shape[1])]
)

# 4. Gộp vào df
df = pd.concat([df.reset_index(drop=True), genre_tfidf_df], axis=1)

df.head()



TF-IDF shape: (10001, 24)


,Poster,Title,Year,Certificate,Duration (min),Genre,Rating,Metascore,Director,Cast,...,genre_vec_14,genre_vec_15,genre_vec_16,genre_vec_17,genre_vec_18,genre_vec_19,genre_vec_20,genre_vec_21,genre_vec_22,genre_vec_23
0,https://m.media-amazon.com/images/M/MV5BYWRkZj...,The Idea of You,2023.0,R,115.0,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...",...,0.0,0.0,0.0,0.0,0.727621,0.000000,0.00000,0.0,0.0,0.0
1,https://m.media-amazon.com/images/M/MV5BZGI4NT...,Kingdom of the Planet of the Apes,2023.0,PG-13,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...",...,0.0,0.0,0.0,0.0,0.000000,0.572247,0.00000,0.0,0.0,0.0
2,https://m.media-amazon.com/images/M/MV5BZjIyOT...,Unfrosted,2023.0,PG-13,97.0,"Biography, Comedy, History",5.5,42.0,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...",...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0
3,https://m.media-amazon.com/images/M/MV5BMjA5Zj...,The Fall Guy,2023.0,PG-13,126.0,"Action, Comedy, Drama",7.3,73.0,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...",...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0
4,https://m.media-amazon.com/images/M/MV5BNTk1MT...,Challengers,2023.0,R,131.0,"Drama, Romance, Sport",7.7,82.0,Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...",...,0.0,0.0,0.0,0.0,0.482113,0.000000,0.83099,0.0,0.0,0.0


#### 4️. Embedding + PCA (Text & People)
- **Columns:** `Description`, `Review Title`, `Review`, `Director`, `Cast`
- **Method:**
  - Fill NaN values with `"Unknown"`.
  - Combine multiple text fields when needed (`text_combined = Description + Review Title + Review` or `people_combined = Director + Cast`).
  - Generate dense embeddings using **SentenceTransformer** (`all-MiniLM-L6-v2`).
  - Reduce embedding dimensions using **PCA** to avoid overfitting and speed up training.
- **Output:** 
  - Text embeddings → 64 dimensions per movie (`text_vec_0 ... text_vec_63`)
  - People embeddings → 32 dimensions per movie (`people_vec_0 ... people_vec_31`)

In [13]:
# --- Embedding DIRECTOR + CAST + PCA ---

import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
import numpy as np

# 1. Làm sạch dữ liệu Director & Cast
df["Director"] = df["Director"].fillna("Unknown")
df["Cast"] = df["Cast"].fillna("Unknown")

# 2. Kết hợp thành 1 chuỗi duy nhất để embedding
df["people_combined"] = df["Director"].astype(str) + " . " + df["Cast"].astype(str)

# 3. Load SentenceTransformer model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 4. Embedding Director+Cast
people_embeddings = model.encode(
    df["people_combined"].tolist(),
    show_progress_bar=True
)
people_embeddings = np.array(people_embeddings)   # shape (N, 384)

print("Raw embedding shape:", people_embeddings.shape)

# 5. PCA giảm chiều xuống 32
pca_people = PCA(n_components=32, random_state=42)
people_pca = pca_people.fit_transform(people_embeddings)

print("PCA reduced shape:", people_pca.shape)

# 6. Đưa vector PCA vào DataFrame
people_pca_df = pd.DataFrame(
    people_pca,
    columns=[f"people_vec_{i}" for i in range(people_pca.shape[1])]
)

df = pd.concat([df.reset_index(drop=True), people_pca_df], axis=1)

df.head()


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Raw embedding shape: (10001, 384)
PCA reduced shape: (10001, 32)


,Poster,Title,Year,Certificate,Duration (min),Genre,Rating,Metascore,Director,Cast,...,people_vec_22,people_vec_23,people_vec_24,people_vec_25,people_vec_26,people_vec_27,people_vec_28,people_vec_29,people_vec_30,people_vec_31
0,https://m.media-amazon.com/images/M/MV5BYWRkZj...,The Idea of You,2023.0,R,115.0,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...",...,-0.034673,-0.105383,0.109582,-0.017727,-0.041650,0.153764,0.034851,-0.056116,0.100694,0.025116
1,https://m.media-amazon.com/images/M/MV5BZGI4NT...,Kingdom of the Planet of the Apes,2023.0,PG-13,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...",...,0.067829,0.113895,0.013765,0.059624,-0.005777,0.017174,-0.024778,-0.207942,0.051285,-0.085777
2,https://m.media-amazon.com/images/M/MV5BZjIyOT...,Unfrosted,2023.0,PG-13,97.0,"Biography, Comedy, History",5.5,42.0,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...",...,-0.133476,-0.066947,-0.099399,-0.043531,-0.081811,-0.033182,0.102137,0.044158,0.082154,0.031845
3,https://m.media-amazon.com/images/M/MV5BMjA5Zj...,The Fall Guy,2023.0,PG-13,126.0,"Action, Comedy, Drama",7.3,73.0,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...",...,-0.038972,-0.043870,-0.054218,-0.047532,0.069378,-0.035428,-0.002614,0.111272,0.039842,-0.032591
4,https://m.media-amazon.com/images/M/MV5BNTk1MT...,Challengers,2023.0,R,131.0,"Drama, Romance, Sport",7.7,82.0,Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...",...,-0.117223,-0.027064,-0.079687,0.116524,-0.013194,0.028850,-0.039236,-0.050396,-0.030880,0.033503


In [14]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA

# 1. Clean missing text fields
df["Description"] = df["Description"].fillna("Unknown")
df["Review"] = df["Review"].fillna("Unknown")
df["Review Title"] = df["Review Title"].fillna("Unknown")

# 2. Combine text fields into 1 string per row
df["text_combined"] = (
    df["Title"].astype(str) + " . " +
    df["Description"].astype(str) + " . " +
    df["Review Title"].astype(str) + " . " +
    df["Review"].astype(str)
)

# 3. Load embedding model
text_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 4. Encode combined text → embedding (N, 384)
text_embeddings = text_model.encode(
    df["text_combined"].tolist(),
    show_progress_bar=True
)
text_embeddings = np.array(text_embeddings)

print("Raw embedding shape:", text_embeddings.shape)

# 5. PCA reduction → (N, 64)
pca_text = PCA(n_components=64, random_state=42)
text_pca = pca_text.fit_transform(text_embeddings)

print("PCA reduced shape:", text_pca.shape)

# 6. Insert PCA vectors back into DataFrame
text_pca_df = pd.DataFrame(
    text_pca,
    columns=[f"text_vec_{i}" for i in range(text_pca.shape[1])]
)

df = pd.concat([df.reset_index(drop=True), text_pca_df], axis=1)

df.head()

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Raw embedding shape: (10001, 384)
PCA reduced shape: (10001, 64)


,Poster,Title,Year,Certificate,Duration (min),Genre,Rating,Metascore,Director,Cast,...,text_vec_54,text_vec_55,text_vec_56,text_vec_57,text_vec_58,text_vec_59,text_vec_60,text_vec_61,text_vec_62,text_vec_63
0,https://m.media-amazon.com/images/M/MV5BYWRkZj...,The Idea of You,2023.0,R,115.0,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...",...,-0.005287,0.067457,-0.070887,-0.020173,-0.051936,0.033685,-0.083173,0.025447,0.049097,0.014704
1,https://m.media-amazon.com/images/M/MV5BZGI4NT...,Kingdom of the Planet of the Apes,2023.0,PG-13,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...",...,-0.016395,-0.076337,-0.023177,-0.005776,-0.045230,-0.027044,0.003682,0.010401,0.010828,-0.051337
2,https://m.media-amazon.com/images/M/MV5BZjIyOT...,Unfrosted,2023.0,PG-13,97.0,"Biography, Comedy, History",5.5,42.0,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...",...,-0.007290,-0.050690,-0.006807,-0.058425,0.026508,0.018325,0.055112,0.016033,0.008481,-0.056422
3,https://m.media-amazon.com/images/M/MV5BMjA5Zj...,The Fall Guy,2023.0,PG-13,126.0,"Action, Comedy, Drama",7.3,73.0,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...",...,0.020255,0.138068,0.098025,0.019770,-0.099873,0.072497,-0.042614,0.119478,0.019714,-0.025817
4,https://m.media-amazon.com/images/M/MV5BNTk1MT...,Challengers,2023.0,R,131.0,"Drama, Romance, Sport",7.7,82.0,Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...",...,0.060469,-0.007018,-0.081993,-0.025112,-0.084943,-0.021575,-0.031951,0.047115,0.080414,0.071885


#### 5. Notes & Considerations
The Metascore column represents aggregated scores from professional critics (out of 100). However, the dataset contains a significant number of missing values (NaN), which need to be addressed before building the recommendation system.

Missing values are handled using predictive modeling.



In [15]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, csr_matrix

# ===========================================================
# 1. Numeric columns
# ===========================================================
NUMERIC_COLS = ["Rating", "Duration (min)", "Votes", "Review Count"]

for col in NUMERIC_COLS:
    df[col] = pd.to_numeric(df[col], errors="coerce")
    df[col] = df[col].fillna(df[col].median())

df_numeric = df[NUMERIC_COLS].astype("float32")


# ===========================================================
# 2. Certificate One-Hot
# ===========================================================
cert_cols = [c for c in df.columns if c.startswith("cert_")]
df_cert = df[cert_cols].astype("float32").fillna(0)


# ===========================================================
# 3. People Embedding (Director + Cast PCA)
# ===========================================================
people_cols = [c for c in df.columns if c.startswith("people_vec_")]
df_people = df[people_cols].astype("float32").fillna(0)


# ===========================================================
# 4. Text Embedding (Title + Description + ReviewTitle + Review PCA)
# ===========================================================
text_cols = [c for c in df.columns if c.startswith("text_vec_")]
df_text = df[text_cols].astype("float32").fillna(0)


# ===========================================================
# 5. Genre TF-IDF
# ===========================================================
df["Genre_clean"] = df["Genre"].fillna("").str.replace(",", " ")

tfidf = TfidfVectorizer()
genre_tfidf = tfidf.fit_transform(df["Genre_clean"])

print("TF-IDF Genre shape:", genre_tfidf.shape)


# ===========================================================
# 6. Combine dense features → sparse
# ===========================================================
features_dense = pd.concat([
    df_numeric.reset_index(drop=True),
    df_cert.reset_index(drop=True),
    df_people.reset_index(drop=True),
    df_text.reset_index(drop=True)
], axis=1)

features_dense = features_dense.fillna(0).astype("float32")
X_dense_sparse = csr_matrix(features_dense.values)

# Dense + Sparse
X_all = hstack([X_dense_sparse, genre_tfidf], format="csr")

print("Final Feature Matrix shape:", X_all.shape)


# ===========================================================
# 7. Split metascore known vs missing
# ===========================================================
mask_known = df["Metascore"].notna().to_numpy()

X_known = X_all[mask_known]
y_known = df.loc[mask_known, "Metascore"].astype("float32").values

X_missing = X_all[~mask_known]


# ===========================================================
# 8. Train-test split
# ===========================================================
X_train, X_test, y_train, y_test = train_test_split(
    X_known, y_known, test_size=0.2, random_state=42
)


# ===========================================================
# 9. Train RandomForest
# ===========================================================
model = RandomForestRegressor(
    n_estimators=200,
    max_depth=30,
    min_samples_split=3,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

print("Train completed.")
print("Training R²:", model.score(X_train, y_train))
print("Test R²:", model.score(X_test, y_test))


# ===========================================================
# 10. Predict missing metascore
# ===========================================================
pred = model.predict(X_missing)

df.loc[~mask_known, "Metascore"] = pred

print("\nMissing after fill:", df['Metascore'].isna().sum())

TF-IDF Genre shape: (10001, 24)
Final Feature Matrix shape: (10001, 155)
Train completed.
Training R²: 0.941267625357434
Test R²: 0.5440785176458185

Missing after fill: 0


# Final

In [16]:
print("Total columns in df:", df.shape[1])
print("Total rows:", df.shape[0])


Total columns in df: 180
Total rows: 10001


In [17]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Columns: 180 entries, Poster to text_vec_63
dtypes: bool(40), float32(96), float64(32), object(12)
memory usage: 7.4+ MB


In [18]:
cols_to_drop = [
    "Poster", "Title", "Certificate", "Genre", "Year",
    "Director", "Cast", "Description", "Review Title", "Review", "text_combined", "people_combined"
]

df = df.drop(columns=cols_to_drop)


In [19]:
cols_to_drop = [
    "Rating_clean", "Duration_int", "Genre_clean"
]

df = df.drop(columns=cols_to_drop)

In [20]:
# Kiểm tra xem DataFrame còn missing values không
missing_counts = df.isna().sum()
total_missing = missing_counts.sum()

print("Missing values per column:")
print(missing_counts[missing_counts > 0])

if total_missing == 0:
    print("\nNo missing values remaining in the DataFrame.")
else:
    print(f"\nTotal missing values in the DataFrame: {total_missing}")


Missing values per column:
Series([], dtype: int64)

No missing values remaining in the DataFrame.


In [21]:
from sklearn.preprocessing import MinMaxScaler

numeric_cols = ["Rating", "Duration (min)", "Votes", "Review Count", "Metascore"]

# Chuẩn hóa 0-1
scaler = MinMaxScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Kiểm tra
df[numeric_cols].head()


,Rating,Duration (min),Votes,Review Count,Metascore
0,0.6375,0.099857,0.009927,0.016472,0.666667
1,0.7500,0.142653,0.007683,0.018158,0.656566
2,0.5250,0.074180,0.006354,0.033042,0.414141
3,0.7500,0.115549,0.013454,0.038103,0.727273
4,0.8000,0.122682,0.011230,0.019250,0.818182


In [22]:
df.head()

,Duration (min),Rating,Metascore,Votes,Review Count,year_1946-1959,year_1960-1969,year_1970-1979,year_1980-1989,year_1990-1999,...,text_vec_54,text_vec_55,text_vec_56,text_vec_57,text_vec_58,text_vec_59,text_vec_60,text_vec_61,text_vec_62,text_vec_63
0,0.099857,0.6375,0.666667,0.009927,0.016472,False,False,False,False,False,...,-0.005287,0.067457,-0.070887,-0.020173,-0.051936,0.033685,-0.083173,0.025447,0.049097,0.014704
1,0.142653,0.7500,0.656566,0.007683,0.018158,False,False,False,False,False,...,-0.016395,-0.076337,-0.023177,-0.005776,-0.045230,-0.027044,0.003682,0.010401,0.010828,-0.051337
2,0.074180,0.5250,0.414141,0.006354,0.033042,False,False,False,False,False,...,-0.007290,-0.050690,-0.006807,-0.058425,0.026508,0.018325,0.055112,0.016033,0.008481,-0.056422
3,0.115549,0.7500,0.727273,0.013454,0.038103,False,False,False,False,False,...,0.020255,0.138068,0.098025,0.019770,-0.099873,0.072497,-0.042614,0.119478,0.019714,-0.025817
4,0.122682,0.8000,0.818182,0.011230,0.019250,False,False,False,False,False,...,0.060469,-0.007018,-0.081993,-0.025112,-0.084943,-0.021575,-0.031951,0.047115,0.080414,0.071885


In [23]:
all_columns = df.columns.tolist()
all_columns

['Duration (min)',
 'Rating',
 'Metascore',
 'Votes',
 'Review Count',
 'year_1946-1959',
 'year_1960-1969',
 'year_1970-1979',
 'year_1980-1989',
 'year_1990-1999',
 'year_2000-2009',
 'year_2010+',
 'year_Pre-1945',
 'year_Unknown',
 'cert_(Banned)',
 'cert_12',
 'cert_12+',
 'cert_13',
 'cert_15+',
 'cert_16',
 'cert_16+',
 'cert_18',
 'cert_18+',
 'cert_7',
 'cert_A',
 'cert_All',
 'cert_Approved',
 'cert_G',
 'cert_GP',
 'cert_M/PG',
 'cert_NC-17',
 'cert_Not Rated',
 'cert_PG',
 'cert_PG-13',
 'cert_R',
 'cert_U',
 'cert_U/A',
 'cert_U/A 16+',
 'cert_UA',
 'cert_UA 13+',
 'cert_UA 16+',
 'cert_UA 7+',
 'cert_Unknown',
 'cert_Unrated',
 'cert_X',
 'genre_vec_0',
 'genre_vec_1',
 'genre_vec_2',
 'genre_vec_3',
 'genre_vec_4',
 'genre_vec_5',
 'genre_vec_6',
 'genre_vec_7',
 'genre_vec_8',
 'genre_vec_9',
 'genre_vec_10',
 'genre_vec_11',
 'genre_vec_12',
 'genre_vec_13',
 'genre_vec_14',
 'genre_vec_15',
 'genre_vec_16',
 'genre_vec_17',
 'genre_vec_18',
 'genre_vec_19',
 'genre_ve

In [24]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# =========================
# Giả sử df hiện tại đã là DataFrame để recommend
# =========================
df = df.reset_index(drop=True)
df['id'] = df.index  # tạo cột id

# =========================
# Xác định các feature
# =========================
feature_cols = [c for c in df.columns if c != 'id']
X = df[feature_cols].values

# =========================
# Đặt trọng số cho từng nhóm feature
# =========================
feature_weights = {
    'Rating': 2.0,
    'Review Count': 1.5,
    'Duration (min)': 0.5,
    'Votes': 1.0,
    'Metascore': 1.0,
    'genre_vec': 1.2,
    'people_vec': 1.0,
    'text_vec': 1.5
}

# Tạo ma trận X_weighted
X_weighted = X.copy()
for i, col in enumerate(feature_cols):
    if col.startswith('genre_vec'):
        X_weighted[:, i] *= feature_weights['genre_vec']
    elif col.startswith('people_vec'):
        X_weighted[:, i] *= feature_weights['people_vec']
    elif col.startswith('text_vec'):
        X_weighted[:, i] *= feature_weights['text_vec']
    else:
        if col in feature_weights:
            X_weighted[:, i] *= feature_weights[col]

# =========================
# Tính cosine similarity
# =========================
similarity_matrix = cosine_similarity(X_weighted)

# Lưu ma trận similarity ra file CSV
sim_df = pd.DataFrame(similarity_matrix, index=df['id'], columns=df['id'])
sim_df.to_csv('movie_similarity_matrix.csv', index=True)
print("Ma trận cosine similarity đã được lưu vào movie_similarity_matrix.csv")

# =========================
# Lấy top 10 phim tương tự cho mỗi phim
# =========================
top_n = 10
recommend_list = []

for i in range(similarity_matrix.shape[0]):
    sim_scores = similarity_matrix[i]
    top_indices = np.argsort(sim_scores)[::-1]  # giảm dần
    top_indices = top_indices[top_indices != i]  # loại bỏ chính phim
    top_indices = top_indices[:top_n]
    recommend_list.append(top_indices.tolist())

# =========================
# Tạo DataFrame recommend
# =========================
df_recommend = pd.DataFrame({
    'id': df['id'],
    'recommended_ids': recommend_list
})

# In ra 5 dòng đầu tiên
print(df_recommend.head())


Ma trận cosine similarity đã được lưu vào movie_similarity_matrix.csv
   id                                    recommended_ids
0   0  [4531, 8432, 4768, 9987, 8249, 9331, 3291, 865...
1   1  [2773, 4376, 6376, 10, 4371, 1876, 7098, 1020,...
2   2  [81, 4696, 4067, 1769, 5490, 1405, 5499, 7447,...
3   3  [5150, 8238, 6355, 6644, 9297, 7312, 2500, 865...
4   4  [9585, 2156, 7923, 7555, 8858, 2928, 3326, 341...


In [25]:
import pandas as pd

# Đọc data gốc
df_raw = pd.read_csv("archive\imdb-movies-dataset.csv")

# Tạo cột id dựa trên thứ tự
df_raw = df_raw.reset_index(drop=True)
df_raw['id'] = df_raw.index

# Merge với df_recommend
df_merged = pd.merge(df_raw, df_recommend, on='id', how='left')

# Sắp xếp lại cột: id, recommended_ids, các cột còn lại
cols = ['id', 'recommended_ids'] + [c for c in df_merged.columns if c not in ['id', 'recommended_ids']]
df_merged = df_merged[cols]

# Lưu ra CSV
df_merged.to_csv("recommend_system.csv", index=False)

print("Hoàn tất. File imdb_movies_with_recommend.csv đã được tạo.")
df_merged.head()


<>:4: SyntaxWarning:

invalid escape sequence '\i'

<>:4: SyntaxWarning:

invalid escape sequence '\i'

C:\Users\Admin\AppData\Local\Temp\ipykernel_36408\1349184853.py:4: SyntaxWarning:

invalid escape sequence '\i'



Hoàn tất. File imdb_movies_with_recommend.csv đã được tạo.


,id,recommended_ids,Poster,Title,Year,Certificate,Duration (min),Genre,Rating,Metascore,Director,Cast,Votes,Description,Review Count,Review Title,Review
0,0,"[4531, 8432, 4768, 9987, 8249, 9331, 3291, 865...",https://m.media-amazon.com/images/M/MV5BYWRkZj...,The Idea of You,2023.0,R,115.0,"Comedy, Drama, Romance",6.4,67.0,Michael Showalter,"Anne Hathaway, Nicholas Galitzine, Ella Rubin,...","28,744","Solène, a 40-year-old single mom, begins an un...",166,Hypocrisy as an idea,"This film, as well as the reaction to it, is a..."
1,1,"[2773, 4376, 6376, 10, 4371, 1876, 7098, 1020,...",https://m.media-amazon.com/images/M/MV5BZGI4NT...,Kingdom of the Planet of the Apes,2023.0,PG-13,145.0,"Action, Adventure, Sci-Fi",7.3,66.0,Wes Ball,"Owen Teague, Freya Allan, Kevin Durand, Peter ...","22,248","Many years after the reign of Caesar, a young ...",183,A phenomenal start to another trilogy!,"I'm a big fan of all the planet of the apes, a..."
2,2,"[81, 4696, 4067, 1769, 5490, 1405, 5499, 7447,...",https://m.media-amazon.com/images/M/MV5BZjIyOT...,Unfrosted,2023.0,PG-13,97.0,"Biography, Comedy, History",5.5,42.0,Jerry Seinfeld,"Isaac Bae, Jerry Seinfeld, Chris Rickett, Rach...","18,401","In 1963 Michigan, business rivals Kellogg's an...",333,not funny,Pretty much the worst criticism you can lay on...
3,3,"[5150, 8238, 6355, 6644, 9297, 7312, 2500, 865...",https://m.media-amazon.com/images/M/MV5BMjA5Zj...,The Fall Guy,2023.0,PG-13,126.0,"Action, Comedy, Drama",7.3,73.0,David Leitch,"Ryan Gosling, Emily Blunt, Aaron Taylor-Johnso...","38,953",A down-and-out stuntman must find the missing ...,384,Everything you needed and more!,Just got out of the Austin premier at SXSW and...
4,4,"[9585, 2156, 7923, 7555, 8858, 2928, 3326, 341...",https://m.media-amazon.com/images/M/MV5BNTk1MT...,Challengers,2023.0,R,131.0,"Drama, Romance, Sport",7.7,82.0,Luca Guadagnino,"Zendaya, Mike Faist, Josh O'Connor, Darnell Ap...","32,517","Tashi, a former tennis prodigy turned coach, t...",194,"Watch ""Match Point"" instead",This is a tough one. I liked the concept and t...
